#### Import Required Libraries

In [0]:
# Library to suppress warnings or deprecation notes
import warnings

warnings.filterwarnings("ignore")

# Libraries to help with reading and manipulating data

import pandas as pd
import numpy as np

# Library to split data
from sklearn.model_selection import train_test_split

# libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 200)

# Libraries to build decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# To tune different models
from sklearn.model_selection import GridSearchCV

# To perform statistical analysis
import scipy.stats as stats

# To get diferent metric scores
from sklearn import metrics

In [0]:
# -------------------------------
# Load data from snowflake and converting it into pandas dataframe
# -------------------------------
path="/Workspace/Users/ankur242199@exlservice.com/Data_Files/"
df = pd.read_csv(path+ "Dow Jones - POC dataset - modified_poc_dataset.csv")
df.head()

##Data Overview

* Observations 
* Sanity Checks

In [0]:
# Converting all columns to upper case
df.columns = [i.upper() for i in df.columns]
df.head()

In [0]:
# -------------------------------
# Checking Min and Max data
# -------------------------------
df['WEEK_START'].min(), df['WEEK_START'].max()

In [0]:
# -------------------------------
# Checking all columns available
# -------------------------------
df.columns

In [0]:
# -------------------------------
# Creating Month and week number variable : If date is greater than equal to 28, month indicator increased by 1 and 
# week number reset to 1 
# -------------------------------
df["WEEK_START"] = pd.to_datetime(df["WEEK_START"])
df["CUSTOM_MONTH"] = (df["WEEK_START"].dt.month 
                      + (df["WEEK_START"].dt.day >= 28)).astype(int)

# Sequential month counter (1, 2, 3, …)
df["MONTH_NUM"] = (df["CUSTOM_MONTH"] != df["CUSTOM_MONTH"].shift()).cumsum()

# --- Step 2: Week counter within each month ---
df["WEEK_NO_IN_MONTH"] = df.groupby("MONTH_NUM").cumcount() + 1
df["CUSTOM_MONTH"] = np.where(df["CUSTOM_MONTH"] == 13, 1, df["CUSTOM_MONTH"])

In [0]:
# -------------------------------
# Checking Min and Max values in above create variables
# -------------------------------
print(df['CUSTOM_MONTH'].min(), df['CUSTOM_MONTH'].max())
print(df['MONTH_NUM'].min(), df['MONTH_NUM'].max())
print(df['WEEK_NO_IN_MONTH'].min(), df['WEEK_NO_IN_MONTH'].max())

# Dropping as not needed 
df = df.drop(columns = ['MONTH_NUM'])

In [0]:
df.describe(percentiles = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.90, 0.95, 0.975, 0.99])

In [0]:
# -------------------------------
# Checking Shape of dataframe and length of media and control variables
# -------------------------------
df.shape

In [0]:
# -------------------------------
# Sorting data based on date column
# -------------------------------
df = df.sort_values('WEEK_START').reset_index(drop = True)

#### Exploratory Data Analysis

In [0]:
# -------------------------------
# Find correlation between media flags and Target variable(orders)
# -------------------------------

var = ['ASA_APP', 'BING_DISPLAY', 'BING_SEARCH', 'DV360_DISPLAY',
       'DV360_DISPLAY_OR_OLV', 'FACEBOOK_SOCIAL', 'GOOGLE_DISPLAY',
       'GOOGLE_DISPLAY_OR_OLV', 'GOOGLE_SEARCH', 'LINKEDIN_SOCIAL', 'META_APP',
       'REDDIT_SOCIAL', 'SNAPCHAT_SOCIAL', 'TAPTICA_APP', 'TWITTER_SOCIAL',
       'COMMISSIONS_AFFILIATE', 'PLACEMENT_AFFILIATE', 'BRAND_SPEND',
       'REDBOX_APP', 'LIFTOFF_APP', 'ORDERS']

corr = df[var].corr()

# plot the heatmap

plt.figure(figsize=(15, 7))
sns.heatmap(corr, annot=True, vmin=-1, vmax=1, fmt=".2f", cmap="Spectral")
plt.show()

**Observations**
* Orders have strong corrolations with GOOGLE_DISPLAY_OR_OLV, GOOGLE_SEARCH and FACEBOOK_SOCIAL

#### Creating Train-Test Split

In [0]:
# -------------------------------
# Parameters
# -------------------------------
total_rows = 184
train_weeks = 130
gap_weeks = 4  
test_weeks = total_rows - (train_weeks + gap_weeks)

# -------------------------------
# Compute split indices
# -------------------------------
train_end = train_weeks
test_start = train_end + gap_weeks

# -------------------------------
# Media variables
# -------------------------------
Train = df.iloc[:train_end].drop_duplicates().reset_index(drop = True)
Test = df.iloc[test_start:].drop_duplicates().reset_index(drop = True)
print("Checking Shapes:", Train.shape, Test.shape)



In [0]:
Train.describe()

In [0]:
Test.describe()

In [0]:
# Save persistently
df.to_csv(path+"POC_Clean_Dataset.csv", index=False)
Train.to_csv(path+"Train_Dataset.csv", index=False)
Test.to_csv(path+"Test_Dataset.csv", index=False)
